In [1]:
!pip install simpletransformers

     |████████████████████████████████| 247 kB 8.5 MB/s 
     |████████████████████████████████| 9.1 MB 53.9 MB/s 
     |████████████████████████████████| 3.3 MB 61.5 MB/s 
     |████████████████████████████████| 298 kB 62.9 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.7 MB 56.2 MB/s 
     |████████████████████████████████| 1.2 MB 69.0 MB/s 
     |████████████████████████████████| 3.1 MB 76.6 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 61 kB 622 kB/s 
     |████████████████████████████████| 596 kB 64.3 MB/s 
     |████████████████████████████████| 140 kB 69.5 MB/s 
     |████████████████████████████████| 180 kB 70.0 MB/s 
     |████████████████████████████████| 97 kB 8.4 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 243 kB 64.2 MB/s 
     |████████████████████████████████| 132 kB 79.8 MB/s 
     |█████████████████

In [3]:
import pandas as pd
from sklearn import preprocessing
from simpletransformers.model import TransformerModel
from sklearn.metrics import f1_score, accuracy_score
import csv

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# EXTRA - Only 4 Labels

In [4]:
with open ('train_selfdialogue_adan.txt', 'r') as in_file:
  stripped = (line.strip() for line in in_file)
  lines = (line.split("\t") for line in stripped if line)
  with open('log.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    #writer.writerow(('Label', 'Text'))
    writer.writerows(lines)

In [5]:
with open ('test_selfdialogue_adan.txt', 'r') as in_file:
  stripped = (line.strip() for line in in_file)
  lines = (line.split("\t") for line in stripped if line)
  with open('log_T.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    #writer.writerow(('Label', 'Text'))
    writer.writerows(lines)

In [6]:
dataset = pd.read_csv('log.csv', header= None)
dataset_T = pd.read_csv('log_T.csv', header= None)

In [7]:
dataset.iloc[:,0]
dataset_T.iloc[:,0]

0         __label__music
1         __label__music
2         __label__music
3         __label__music
4         __label__music
              ...       
73595    __label__sports
73596    __label__sports
73597    __label__sports
73598    __label__sports
73599    __label__sports
Name: 0, Length: 73600, dtype: object

In [8]:
dataset[["Label"]] = dataset.iloc[:,0]
dataset[["Text"]] = dataset.iloc[:,2]

In [9]:
dataset_T[["Label"]] = dataset_T.iloc[:,0]
dataset_T[["Text"]] = dataset_T.iloc[:,2]

In [10]:
dataset = dataset.drop(dataset.iloc[:,0:3], axis=1)
dataset_T = dataset_T.drop(dataset_T.iloc[:,0:3], axis=1)

In [11]:
le = preprocessing.LabelEncoder()
le.fit(dataset["Label"])

LabelEncoder()

In [12]:
dataset['Label'] = le.transform(dataset["Label"])
dataset_T['Label'] = le.transform(dataset_T["Label"])

In [13]:
dataset.Label.value_counts()

1    123558
2    105204
3     62478
0      3164
Name: Label, dtype: int64

In [14]:
train_df = dataset[["Text", "Label"]]
eval_df = dataset_T[["Text", "Label"]]

In [15]:
eval_df

,Text,Label
0,Yeah I know I hate the traffic but love all th...,2
1,Do you know who's giving free concerts,2
2,I think Lone Stars Keith Someone the one marri...,2
3,Yeah I know who yo mean,2
4,And Maybe Justin Aldeen,2
...,...,...
73595,yeah be ready to see that clip again and again,3
73596,I don't mind I can watch it all day,3
73597,this guy is crazy he is going to go a long way,3
73598,I need to draft him for my team,3


In [16]:
train_df = train_df.head(100000)
eval_df = eval_df.head(10000)

In [17]:
model = TransformerModel('roberta', 'roberta-large', num_labels=4, use_cuda=True,
args={'fp16': True, 'fp16_opt_level': 'O1', 'max_seq_length': 128, 
      'learning_rate':1e-5, 'num_train_epochs': 1, 'gradient_accumulation_steps': 1,
      'train_batch_size': 32, 'eval_batch_size': 32, 'weight_decay': 0,               
      'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

In [18]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/3125 [00:00<?, ?it/s]

(3125, 0.4717329715013504)

In [19]:
def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/10000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/313 [00:00<?, ?it/s]

In [20]:
result

{'mcc': 0.7232581996365216,
 'f1': 0.8238,
 'acc': 0.8238,
 'eval_loss': 0.40617112696361235}

In [3]:
'''
from sklearn.metrics import classification_report

def class_report(labels, preds):
      return classification_report(labels, preds, average='micro')


class_report, model_outputs, wrong_predictions = model.eval_model(test_df, report = classification_report)
'''

"\nfrom sklearn.metrics import classification_report\n\ndef class_report(labels, preds):\n      return classification_report(labels, preds, average='micro')\n\n\nclass_report, model_outputs, wrong_predictions = model.eval_model(test_df, report = classification_report)\n"

In [ ]:
#class_report

In [ ]:
predictions, raw_outputs = model.predict(['Watch it again and again'])

In [ ]:
predictions

In [ ]:
predictions = le.inverse_transform(predictions)

In [2]:
predictions

"array(['__label__movie'], dtype=object)"